# A simple agent-based model

An interactive ABM

In [41]:
!pip install tornado==4.5.1 --upgrade

  Found existing installation: tornado 4.3
    Uninstalling tornado-4.3:
      Successfully uninstalled tornado-4.3


abce 0.9.5b0 has requirement tornado==4.3, but you'll have tornado 4.5.1 which is incompatible.


In [1]:
!pip install git+https://github.com/AB-CE/abce.git@icis --upgrade --no-dependencies


  Cloning https://github.com/AB-CE/abce.git (to revision icis) to c:\users\dom\appdata\local\temp\pip-req-build-ahevxcoj
  Running setup.py bdist_wheel for abce: started
  Running setup.py bdist_wheel for abce: finished with status 'done'
  Stored in directory: C:\Users\Dom\AppData\Local\Temp\pip-ephem-wheel-cache-6itlkpra\wheels\c0\5f\56\663c84820b723287ec0eb3675eccc1f08f4a85b42cc43182e4
Successfully built abce
  Found existing installation: abce 0.9.5b0
    Uninstalling abce-0.9.5b0:
      Successfully uninstalled abce-0.9.5b0


In [2]:
import abce

from interactiveABM.firm import Firm
from interactiveABM.people import People

params = dict(
    population=1000,
    people_money=1000,
    num_firms=20,
    firm_money=2000,

    num_days=2000,

    l=0.5,  # constant from CS equation

    num_days_buffer=10,  # number of days worth of wages a firm will keep after giving profits

    phi_upper=10,  # phi_upper * demand gives upper bound to inventory
    phi_lower=2,
    excess=1.1,  # if number of workers offered to work for firm exceeds 110% of ideal number, decrease wage
    wage_increment=0.01,
    price_increment=0.01,
    worker_increment=0.01,
    productivity=1,
    wage_acceptance=1)
simulation = abce.Simulation(name='economy', processes=1)
group_of_firms = simulation.build_agents(Firm, "firm", number=params["num_firms"], **params)
people = simulation.build_agents(People, "people", number=1, **params)


for r in range(params["num_days"]):
    simulation.time = r

    group_of_firms.panel_log(variables=['wage', 'ideal_num_workers'], goods=['workers'])
    people.create_labor()

    vacancies_list = list(group_of_firms.publish_vacencies())

    people.send_workers(vacancies_list)

    group_of_firms.production()
    group_of_firms.pay_workers()
    group_of_firms.pay_dividents()
    group_of_firms.send_prices()
    people.get_prices()
    demand = people.buy_goods()

    group_of_firms.sell_goods()
    group_of_firms.determine_bounds(demand=list(demand)[0])
    (group_of_firms + people).print_possessions()
    group_of_firms.determine_wage()
    group_of_firms.expand_or_change_price()
    (people + group_of_firms).destroy_unused_labor()
    people.consumption()
    group_of_firms.determine_profits()

print('done')

simulation.graph()
path = simulation.path()
simulation.finalize()


import pandas as pd
pd.read_csv(path + "/data.csv")


Round0
    people{'money': 9.094947017729282e-13, 'workers': 500.0, 'produce': 299.99999999999994}
Round1
    people{'money': 9.094947017729282e-13, 'workers': 499.1200031933682, 'produce': 248.6020186903116}
Round2
    people{'money': 1.8189894035458565e-12, 'workers': 496.99677355835496, 'produce': 248.31083160835087}
Round3
    people{'money': 0.0, 'workers': 496.99677355835496, 'produce': 247.0995472874168}
Round4
    people{'money': 0.0, 'workers': 496.99677355835496, 'produce': 245.68999414465856}
Round5
    people{'money': 0.0, 'workers': 496.99677355835496, 'produce': 244.3343333872482}
Round6
    people{'money': 0.0, 'workers': 496.99677355835496, 'produce': 243.17489000969755}
Round7
    people{'money': 1.8189894035458565e-12, 'workers': 496.99677355835496, 'produce': 242.09291731205153}
Round8
    people{'money': 0.0, 'workers': 496.99677355835496, 'produce': 240.85119367293592}
Round9
    people{'money': 0.0, 'workers': 496.99677355835496, 'produce': 239.64180020006162}
Rou

WARNING flexx.webruntime: Given browser 'x' not valid/available;
Falling back to the default browser.


INFO flexx.app: Starting Flexx event loop.


RuntimeError: IOLoop is already running

INFO flexx.app: New session ABCE zLjRprXTKnkxP8WRMfhNLENh


ERROR flexx.app: JS - Uncaught TypeError: Cannot read property 'plot_canvas_view' of null
TypeError: Cannot read property 'plot_canvas_view' of null
    at flexx.classes.BokehWidget.<anonymous> (http://localhost:49190/ABCE/?session_id=zLjRprXTKnkxP8WRMfhNLENh:289:28)
ERROR flexx.app: JS - Uncaught TypeError: Cannot read property 'plot_canvas_view' of null
TypeError: Cannot read property 'plot_canvas_view' of null
    at flexx.classes.BokehWidget.<anonymous> (http://localhost:49190/ABCE/?session_id=zLjRprXTKnkxP8WRMfhNLENh:289:28)
ERROR flexx.app: JS - Uncaught TypeError: Cannot read property 'plot_canvas_view' of null
TypeError: Cannot read property 'plot_canvas_view' of null
    at flexx.classes.BokehWidget.<anonymous> (http://localhost:49190/ABCE/?session_id=zLjRprXTKnkxP8WRMfhNLENh:289:28)
ERROR flexx.app: JS - Uncaught TypeError: Cannot read property 'plot_canvas_view' of null
TypeError: Cannot read property 'plot_canvas_view' of null
    at flexx.classes.BokehWidget.<anonymous> (h

In [3]:
print(simulation.path)

C:\Users\Dom/result/economy_2018-07-06_16-53


In [3]:
pd.read_csv(path + "/data.csv")

FileNotFoundError: File b'C:\\Users\\Dom/result/economy_2018-07-06_16-04/data.csv' does not exist